# ETL for Census data 

## Define the API call

In [1]:
# Import U.S. Census API Key
from config import api_key
import pandas as pd
from census import Census

# Create instances of the Census library for different years
c_2022 = Census(api_key, year=2022)

In [2]:
# Define a list of states with their state codes
states = [
    {"state": "01", "state_abbr": "AL"},  # Alabama
    {"state": "02", "state_abbr": "AK"},  # Alaska
    {"state": "04", "state_abbr": "AZ"},  # Arizona
    {"state": "05", "state_abbr": "AR"},  # Arkansas
    {"state": "06", "state_abbr": "CA"},  # California
    {"state": "08", "state_abbr": "CO"},  # Colorado
    {"state": "09", "state_abbr": "CT"},  # Connecticut
    {"state": "10", "state_abbr": "DE"},  # Delaware
    {"state": "11", "state_abbr": "DC"},  # District of Columbia
    {"state": "12", "state_abbr": "FL"},  # Florida
    {"state": "13", "state_abbr": "GA"},  # Georgia
    {"state": "15", "state_abbr": "HI"},  # Hawaii
    {"state": "16", "state_abbr": "ID"},  # Idaho
    {"state": "17", "state_abbr": "IL"},  # Illinois
    {"state": "18", "state_abbr": "IN"},  # Indiana
    {"state": "19", "state_abbr": "IA"},  # Iowa
    {"state": "20", "state_abbr": "KS"},  # Kansas
    {"state": "21", "state_abbr": "KY"},  # Kentucky
    {"state": "22", "state_abbr": "LA"},  # Louisiana
    {"state": "23", "state_abbr": "ME"},  # Maine
    {"state": "24", "state_abbr": "MD"},  # Maryland
    {"state": "25", "state_abbr": "MA"},  # Massachusetts
    {"state": "26", "state_abbr": "MI"},  # Michigan
    {"state": "27", "state_abbr": "MN"},  # Minnesota
    {"state": "28", "state_abbr": "MS"},  # Mississippi
    {"state": "29", "state_abbr": "MO"},  # Missouri
    {"state": "30", "state_abbr": "MT"},  # Montana
    {"state": "31", "state_abbr": "NE"},  # Nebraska
    {"state": "32", "state_abbr": "NV"},  # Nevada
    {"state": "33", "state_abbr": "NH"},  # New Hampshire
    {"state": "34", "state_abbr": "NJ"},  # New Jersey
    {"state": "35", "state_abbr": "NM"},  # New Mexico
    {"state": "36", "state_abbr": "NY"},  # New York
    {"state": "37", "state_abbr": "NC"},  # North Carolina
    {"state": "38", "state_abbr": "ND"},  # North Dakota
    {"state": "39", "state_abbr": "OH"},  # Ohio
    {"state": "40", "state_abbr": "OK"},  # Oklahoma
    {"state": "41", "state_abbr": "OR"},  # Oregon
    {"state": "42", "state_abbr": "PA"},  # Pennsylvania
    {"state": "44", "state_abbr": "RI"},  # Rhode Island
    {"state": "45", "state_abbr": "SC"},  # South Carolina
    {"state": "46", "state_abbr": "SD"},  # South Dakota
    {"state": "47", "state_abbr": "TN"},  # Tennessee
    {"state": "48", "state_abbr": "TX"},  # Texas
    {"state": "49", "state_abbr": "UT"},  # Utah
    {"state": "50", "state_abbr": "VT"},  # Vermont
    {"state": "51", "state_abbr": "VA"},  # Virginia
    {"state": "53", "state_abbr": "WA"},  # Washington
    {"state": "54", "state_abbr": "WV"},  # West Virginia
    {"state": "55", "state_abbr": "WI"},  # Wisconsin
    {"state": "56", "state_abbr": "WY"},  # Wyoming
    {"state": "60", "state_abbr": "AS"},  # American Samoa
    {"state": "66", "state_abbr": "GU"},  # Guam
    {"state": "69", "state_abbr": "MP"},  # Northern Mariana Islands
    {"state": "72", "state_abbr": "PR"},  # Puerto Rico
    {"state": "78", "state_abbr": "VI"},  # U.S. Virgin Islands
]

# Initialize an empty list to store dataframes for each state
state_dfs = []

# Loop over each state in the list
for state_info in states:

    # Run Census Search to retrieve data for a specific state and year
    census_data_2022 = c_2022.acs5.get(
        (
            "NAME",
            "B19013_001E",
            "B01002_001E",
            "B19301_001E",
        ),
        {'for': 'county:*', 'in': f'state:{state_info["state"]}'}  
    )

    # Convert to DataFrame
    census_pd_state = pd.DataFrame(census_data_2022)

    # Add year and state_abbr information to each record
    census_pd_state['year'] = 2022
    census_pd_state['state_abbr'] = state_info['state_abbr']

    # Rename columns
    census_pd_state = census_pd_state.rename(
        columns={
            "B01002_001E": "median_age",
            "B19013_001E": "average_household_income",
            "B19301_001E": "per_capita_income",
            "NAME": "Name",
            "state": "state_fips",
            "county": "county_fips"
        }
    )

    # Add state dataframe to the list
    state_dfs.append(census_pd_state)

# Concatenate all state dataframes into one dataframe
census_pd_state_all = pd.concat(state_dfs, ignore_index=True)

# Display DataFrame length and sample data
print(f"Number of rows in the DataFrame: {len(census_pd_state_all)}")

# Display the final DataFrame
census_pd_state_all.head()

Number of rows in the DataFrame: 3222


,Name,average_household_income,median_age,per_capita_income,state_fips,county_fips,year,state_abbr
0,"Autauga County, Alabama",68315.0,39.0,35332.0,01,001,2022,AL
1,"Baldwin County, Alabama",71039.0,43.7,38907.0,01,003,2022,AL
2,"Barbour County, Alabama",39712.0,40.6,23378.0,01,005,2022,AL
3,"Bibb County, Alabama",50669.0,40.3,24966.0,01,007,2022,AL
4,"Blount County, Alabama",57440.0,40.8,29053.0,01,009,2022,AL


## Data Cleaning

In [3]:
# Assign the category and subcategory values to category and subcategory columns.
split_columns = census_pd_state_all['Name']\
        .str.split(",", n=1, expand = True)
census_pd_state_all['county_name'] = split_columns[0]
census_pd_state_all['state_name'] = split_columns[1]

# Drop the contact_name column
census_pd_state_all = census_pd_state_all.drop(['Name'], axis = 1)

# Configure the final DataFrame
census_pd_state_all = census_pd_state_all[
    [
        "state_name",
        "state_abbr",
        "county_name",
        "state_fips",
        "county_fips",
        "year",
        "median_age",
        "average_household_income",
        "per_capita_income",
    ]
]

# Display the DataFrame
census_pd_state_all.head()

,state_name,state_abbr,county_name,state_fips,county_fips,year,median_age,average_household_income,per_capita_income
0,Alabama,AL,Autauga County,01,001,2022,39.0,68315.0,35332.0
1,Alabama,AL,Baldwin County,01,003,2022,43.7,71039.0,38907.0
2,Alabama,AL,Barbour County,01,005,2022,40.6,39712.0,23378.0
3,Alabama,AL,Bibb County,01,007,2022,40.3,50669.0,24966.0
4,Alabama,AL,Blount County,01,009,2022,40.8,57440.0,29053.0


In [4]:
# Add census information of missing county from previous year
new_row = {'state_name': ' Connecticut', 'state_abbr': 'CT', 
           'county_name': 'Fairfield County', 'state_fips': '09', 
           'county_fips': '001', 'year': '2022', 'median_age': 40.7,
           'average_household_income': 101194.0, 'per_capita_income': 61651
}
census_pd_state_all.loc[len(census_pd_state_all)] = new_row

census_pd_state_all.sort_values(by=['state_fips', 'county_fips'])

# Display the DataFrame
census_pd_state_all.head()

,state_name,state_abbr,county_name,state_fips,county_fips,year,median_age,average_household_income,per_capita_income
0,Alabama,AL,Autauga County,01,001,2022,39.0,68315.0,35332.0
1,Alabama,AL,Baldwin County,01,003,2022,43.7,71039.0,38907.0
2,Alabama,AL,Barbour County,01,005,2022,40.6,39712.0,23378.0
3,Alabama,AL,Bibb County,01,007,2022,40.3,50669.0,24966.0
4,Alabama,AL,Blount County,01,009,2022,40.8,57440.0,29053.0


In [5]:
# Check for null values
census_pd_state_all.isnull().sum()

state_name                  0
state_abbr                  0
county_name                 0
state_fips                  0
county_fips                 0
year                        0
median_age                  0
average_household_income    0
per_capita_income           0
dtype: int64

## Export the Data

In [6]:
# Export the City_Data into a csv
census_pd_state_all.to_csv("../static/data/census_data_2022.csv", encoding="utf-8", index=False)